In [ ]:
import sys
sys.path.append('../src/')

import numpy as np
from scipy import stats
from scipy.stats.mstats import winsorize
import pandas as pd
import data_io
import utils
import matplotlib.pyplot as plt

#### 1. Read in dataset

In [ ]:
print(str(data_io.input_cleaned))
EXCEL_OPTIONS = {'strings_to_urls': False,
                 'strings_to_formulas': False}

In [ ]:
import os
if os.path.exists(data_io.input_cleaned/'gfm'/'cancer_w_locs_and_text_features_census.xlsx'):
    merge_ndi = False
    print('path exists')
    df = pd.read_excel(data_io.input_cleaned/'gfm'/'cancer_w_locs_and_text_features_census.xlsx', 
                       encoding = 'utf-8')
else:
    print('still need to merge GFM and Census data')
    df = pd.read_excel(data_io.input_cleaned/'gfm'/'cancer_w_locs_and_text_features.xlsx', 
                       encoding = 'utf-8')

#### Format variables for analysis

In [ ]:
if 'winsorized_goal' not in df.columns.values:
    #citation to support winsorizing https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5548942/#!po=41.6667
    df['winsorized_goal'] = winsorize(df['goal_amnt'], limits = .05)
    df.loc[pd.isnull(df['goal_amnt']), 'winsorized_goal'] = np.nan
    print(df['winsorized_goal'].mean())
    print(df['goal_amnt'].mean())
    df['changed_goal'] = df['winsorized_goal'] != df['goal_amnt']
    df.loc[pd.isnull(df['winsorized_goal']), 'changed_goal'] = False
    print('Number of campaigns that were changed in winsorization: ', df.changed_goal.value_counts())
    df['orig_percent_goal'] = df['raised_amnt']/df['goal_amnt']
else:
    print('winsorization already done')


In [ ]:
if 'percent_goal' not in df.columns.values:
    df['percent_goal'] = df['raised_amnt']/df['winsorized_goal']
    print('Number of campaigns with goal amount > 100%: ', len(df[df['percent_goal']>1.]))
    df.loc[df['orig_percent_goal']>1., 'orig_percent_goal'] = 1.
    df.loc[df['percent_goal']>1., 'percent_goal'] = 1.
    df['orig_met_goal'] = df['orig_percent_goal'] >=1.
    df['met_goal'] = df['percent_goal'] >=1.
    print('Number of campaigns meeting goal pre-transform: ', df['orig_met_goal'].value_counts())
    print('Number of campaigns meeting goal post-transform: ',df['met_goal'].value_counts())
else:
    print('percent goal already calculated')

In [ ]:

df['log_raised_amnt'] = np.log(df['raised_amnt'])
df.loc[np.isinf(df['log_raised_amnt']), 'log_raised_amnt'] = np.nan

df['avg_contribution'] = df['raised_amnt']/df['num_contributors']
df.loc[np.isinf(df['avg_contribution']), 'avg_contribution'] = np.nan

In [ ]:
#Convert location info to strings
df['state_fips'] = df['state_x'].map(utils.state_fips)
df['county_fips'] = df['county_fips_int'].apply(utils.format_county_fip)

df['state_county_fips_str'] = df['state_fips'] + df['county_fips']
df['state_county_fips_str'].isnull().sum()

#### 2. After running get_census_data.ipynb and clustr.R, calculate the Neighborhood Deprivation Index and merge with the GFM dataset

In [ ]:
#after running the PCA in R on the census variables, load the dataset back in to compute the deprivation index
def get_ndi_quartiles():
    pca = pd.read_csv(data_io.input_cleaned/'census'/'census_w_factor_weights.csv')
    census = pd.read_csv(data_io.input_cleaned/'census'/'acs_five_year_est.csv')
    census['state_county_fips_str'] = census['state_county_fips_str'].apply(utils.format_state_county_fip)
    pca['state_county_fips_str'] = pca['state_county_fips'].apply(utils.format_state_county_fip)

    colnames = pca.columns.to_list()
    cols_to_sum = [i for i in colnames if 'weighted' in i]

    #pca = pca[temp_cols]
    print(colnames)
    pca['sum'] = 0
    for c in cols_to_sum:
        pca['sum'] += pca[c]

    cols_to_sum.append('state_county_fips_str')
    cols_to_sum.append('sum')
    pca = pca[cols_to_sum]
    from sklearn import preprocessing
    pca['standardized_ndi'] = preprocessing.scale(pca['sum'])
    pca = pd.merge(pca, census, how='left', on='state_county_fips_str')
    
    QUANTS = (np.quantile(pca['standardized_ndi'].values, [.25, .5, .75]))

    def _get_quantile(x):
        if x <= QUANTS[0]:
            return 1
        if x > QUANTS[0] and x <= QUANTS[1]:
            return 2
        if x > QUANTS[1] and x <= QUANTS[2]:
            return 3
        if x > QUANTS[2]:
            return 4
    cols_to_drop = [c for c in pca.columns.to_list() if 'weighted' in c]
    print(cols_to_drop)
    pca['ndi_quantile'] = pca['standardized_ndi'].apply(_get_quantile)
    
    pca.drop(columns = cols_to_drop, inplace=True)
    return pca

### Generate Census stats by NDI quartiles

In [ ]:
#Descriptives for PCA/census vars
if merge_ndi:
    ndi_df = get_ndi_quartiles()
    ndi_map = {1: 'NDI Quartile 1 (least deprived)',
                2: 'NDI Quartile 2',
                3: 'NDI Quartile 3',
                4: 'NDI Quartile 4 (most deprived)'
              }

    census_vars = pd.read_csv(data_io.input_cleaned/'census'/'census_variables.csv')
    census_var_dict = dict(zip(census_vars['variable_label'].to_list(), census_vars['pretty_label'].to_list()))

    census_var_dict['percent_single_parent'] = '% With Single-Parent Households'
    census_var_dict['percent_less_35k'] = '% of Households with Annual Income < $35k'
    ndi_df['NDI Quartile'] = ndi_df['ndi_quantile'].map(ndi_map)

    vars_to_plot = ['unemployment_rate', 'percent_poverty', 'no_health_insurance', 
                    'ed_percent_highschool', 'has_internet', 'percent_single_parent', 
                    'percent_less_35k']
    fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8)) = plt.subplots(4, 2, figsize = (10,12))
    ax_array = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8]


    for i in range(0, len(vars_to_plot)):
        if i == 1:
            ndi_df.pivot(columns="NDI Quartile", values=vars_to_plot[i]).plot.hist(bins=100, ax = ax_array[i])
            ax_array[i].legend(loc = 'center left', bbox_to_anchor = (1, .5))
        else:
            ndi_df.pivot(columns="NDI Quartile", values=vars_to_plot[i]).plot.hist(bins=100, ax = ax_array[i],
                                                                                      legend = False)
        xlab = census_var_dict[vars_to_plot[i]]
        if '%' not in xlab:
            xlab += ' (%)'
        ax_array[i].set_xlabel(xlab)
        ptitle = f'{xlab}'
        ax_array[i].set_title(ptitle)
        
    plt.suptitle('Socioeconomic Status Variables by NDI Quartile', y = 1.01, x = 0.4)
    fig.subplots_adjust(hspace=0.5)
    plt.tight_layout()
    plt.savefig(data_io.output_plots/'ndi_comparisons.png', dpi = 400, bbox_inches = 'tight')
    plt.show()
else:
    print('skipping NDI summary')

### Merge Census Stats with GFM dataset

In [ ]:
#check if NDI quartiles are in the dataset
test = [i for i in df.columns.values if 'ndi_quantile' in i]
merged_complete = True if len(test) > 0 else False
print(merged_complete)
#if NDI quartiles aren't already in GFM dataset, add them here:
if merged_complete == False:

    #Get the NDI quartiles--Note: Must run clustr.R to obtain them!
    quarts = get_ndi_quartiles()
    print(len(df))

    #Merge with GFM dataset
    df = pd.merge(df, quarts, how='left', on='state_county_fips_str')
    print(len(df))
    df.rename(columns={'county_x':'county_name', 'county_y':'county_fips_int'}, inplace=True)
    print(df['county_fips_int'].isnull().sum())

#### If the combined dataset hasn't been saved yet, set save to True

In [ ]:

if merge_ndi:
    writer = pd.ExcelWriter(data_io.input_cleaned/'gfm'/'cancer_w_locs_and_text_features_census.xlsx',
                           engine='xlsxwriter', options = EXCEL_OPTIONS)
    df.to_excel(writer, encoding='utf-8', index= False)
    writer.close()
else:
    print('df already saved')

In [ ]:
ndi_map = {1: '1 (least deprived)', 
           2: '2',
           3: '3',
           4: '4 (most deprived)'
          }

#convert NDI quartiles to factors (instead of integers)
df['ndi_quantile_f'] = df['ndi_quantile'].map(ndi_map)
vals = df['ndi_quantile_f'].value_counts()
for v in vals.index:
    print(v)
    print(vals[v])
    print(vals[v]/len(df))
    


### Univariate Analysis

In [ ]:
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.formula.api import ols
from statsmodels.formula.api import glm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

#### Define functions for univariate analysis

In [ ]:

def format_percent(x, digits = 2):
    if type(x) != str and pd.isnull(x)==False:
        if digits == 2:
            x = '{:.2%}'.format(x)
        else:
            x = '{:.3%}'.format(x)
    return x

def format_decimal(x, digits = 2):
    if type(x) != str and type(x) != bool and pd.isnull(x)==False:
        if digits == 2:
            x = '{0:,.2f}'.format(x)
        else:
            x = '{0:,.3f}'.format(x)
        return x
    else:
        return x
    
def format_mean_sd(mean, sd):
    mean = format_decimal(mean)
    sd = format_decimal(sd)
    
    return f"${mean} (${sd})"
    


#Run an anova
def compare_info_by_group(df, comp_cols, group_col):
    for c in range(0, len(comp_cols)):
        temp = df[~pd.isnull(df[comp_cols[c]])]
        new = temp.groupby(group_col[c])
        temp = temp[~pd.isnull(temp[group_col[c]])]
        
        mc = MultiComparison(temp[comp_cols[c]], temp[group_col[c]])
        
        result = mc.tukeyhsd()
        mc_result = result.summary()
        results_as_html = mc_result.as_html()
        result_df = pd.read_html(results_as_html, header=0)[0]
        result_df
        return result_df
    
    
    

In [ ]:
#Set columns for comparisons with labels for plotting
mention_cols = ['brave', 'nice', 'thank', 'self_reliance', 'battle', 'tx_type', 'oop_type', 'insurance_type',
               'ctype']

groups = [m+'_is_mentioned' for m in mention_cols]

#for labeling multivariate outputs
pretty_groups = ['Bravery ', 'Warmth ','Gratitude ', 'Self-reliance ', 'Militaristic Metaphors ',
                'Treatment ', 'OOP Cost ', 'Insurance ', 'Cancer Type ']
#for labeling univariate outputs
pretty_groups_univ = [i.rstrip() for i in pretty_groups]
print(len(groups))
print(len(pretty_groups))
group_label_dict_univ = dict(zip(groups, pretty_groups_univ))
group_label_dict = {}

#### Run Chi-sq tests to compare mention of indicators by NDI quartile

In [ ]:
from scipy.stats import chi2_contingency
#Run pairwise comparisons for Chi-2 test
def get_chi_2_post_hoc(df, groups):
    all_results = pd.DataFrame()
    for g in groups:
        ct = pd.crosstab(df[g], df['ndi_quantile_f'])
        ct.reset_index(inplace= True)
        ct['var'] = g
        ct.sort_values(by = g, inplace=True, ascending = False)
        ct['var_value'] = ['True', 'False']
        ct.drop(columns = [g], inplace=True)
        comp_1_2 = ct[['1 (least deprived)', '2']]
        comp_1_3 = ct[['1 (least deprived)', '3']]
        comp_1_4 = ct[['1 (least deprived)', '4 (most deprived)']]
        comp_2_3 = ct[['2', '3']]
        comp_2_4 = ct[['2', '4 (most deprived)']]
        comp_3_4 = ct[['3', '4 (most deprived)']]
        
        comp_arr = [comp_1_2, comp_1_3, comp_1_4, comp_2_3, comp_2_4, comp_3_4]
        comp_strs = ['comp_1_2', 'comp_1_3', 'comp_1_4', 'comp_2_3', 'comp_2_4', 'comp_3_4']
        for i in range(0, len(comp_arr)):
            chi2, p, dof, ex = (chi2_contingency(comp_arr[i]))
            #Bonferoni correction: 
            min_p = 0.05/len(comp_strs)
            if p < min_p:
                ct.loc[0, f'{comp_strs[i]}_sig_diff'] = True
            else:
                ct.loc[0, f'{comp_strs[i]}_sig_diff'] = False
            ct.loc[0, f'chi_2_{comp_strs[i]}'] = chi2
            ct.loc[0, f'dof_{comp_strs[i]}'] = dof
            ct.loc[0, f'p_val_{comp_strs[i]}'] = p
            
        all_results = all_results.append(ct, ignore_index = True)
        
    return all_results
    
    


In [ ]:

#Run omnibus chi_2 tests for NDI
def calculate_chi_2_ndi(df, g, group_label_dict):
    ndi_quants = ['1 (least deprived)', '2', '3', '4 (most deprived)']
    ct = pd.crosstab(df[g], df['ndi_quantile_f'])
    temp_idx = ct.index.astype(str).to_list()        
    chi2, p, dof, ex = (chi2_contingency(ct))
    ct.set_index(ct.index.astype(str)+f'_{g}', inplace=True)
    
    ct.loc[f'expected_{temp_idx[0]}_{g}', :] = ex[0, :]
    
    ct.loc[f'expected_{temp_idx[1]}_{g}', :] = ex[1, :]
    
    ct.loc[f'expected_less_obs_{g}', :] = ct.loc[f'expected_False_{g}', :] - ct.loc[f'False_{g}', :]
    for nq in ndi_quants:
        ct.loc[f'percent_mentioned_{g}', nq] = format_percent(ct.loc[f'True_{g}', nq] / (ct.loc[f'True_{g}', nq] + 
                                                                          ct.loc[f'False_{g}', nq]))
        
        ct.loc[f'expected_percent_mentioned_{g}', nq] = format_percent(ct.loc[f'expected_True_{g}', nq] / 
                                                                 (ct.loc[f'True_{g}', nq] + ct.loc[f'False_{g}', nq]))
    idx_list = ct.index.to_list()
    pretty_labels = []
    for i in idx_list:
        if 'expected' in i and 'less' not in i:
            if 'expected_percent' in i:
                pretty_labels.append('Expected Percent Mentioned')
            elif 'False' in i:
                pretty_labels.append('Expected False')
            else:
                pretty_labels.append('Expected True')
        else:
            if 'False' in i:
                pretty_labels.append('Observed False')
            elif 'True' in i:
                pretty_labels.append('Observed True')
            elif 'less' in i:
                pretty_labels.append('Expected True - Observed True')
            else:
                pretty_labels.append(' ')

    ct['var'] = group_label_dict[g]
    ct['chi_2'] = chi2
    ct['p'] = p
    ct['df'] = dof
    ct['pretty_labels'] = pretty_labels
    
    return ct
        
        
ndi_quants = ['1 (least deprived)', '2', '3', '4 (most deprived)']
temp_groups = groups.copy()
#first, get the omnibus chi-square tests
all_comps = pd.DataFrame()
for g in temp_groups:
    print(g)
    #get the omnibus
    temp_ct = calculate_chi_2_ndi(df, g, group_label_dict_univ)
    all_comps = all_comps.append(temp_ct)
    
all_comps.reset_index(inplace=True)
#next, get the post-hoc comparisons
post_hoc = get_chi_2_post_hoc(df, groups)


In [ ]:
all_comps

In [ ]:
def unpack_percent(x):
    if type(x) == str:
        x = x[0:-1]
    return float(x)


In [ ]:
#This function helps visualize X2 test results
def plot_chi_2(chi_2, group):
    ndi_quants = ['1 (least deprived)', '2', '3', '4 (most deprived)']
    
    temp_chi = chi_2[chi_2['index'].str.contains(group)]
    temp_chi = temp_chi[temp_chi['index'].str.contains('percent')]
    temp_chi = temp_chi.set_index('index')
    temp_list = []
    for nq in ndi_quants:
        temp_list.append(unpack_percent(temp_chi.loc[f'percent_mentioned_{group}', nq]))
    bars = temp_list
    hline = unpack_percent(temp_chi.loc[f'expected_percent_mentioned_{group}', ndi_quants[0]])
    title = temp_chi.loc[f'expected_percent_mentioned_{group}', 'var'] + ' is Mentioned:\nObserved vs. Expected by NDI Quartile'
    fig, ax = plt.subplots()
    ax.bar(ndi_quants, bars, fc = '#A2D1CE', edgecolor = 'w')
    ax.axhline(hline, linestyle = 'dashed', label = 'Overall % Mentioned', c = 'k')
    ax.set_title(title)
    ax.set_ylim(bottom = 0, top = hline + 3)
    ax.set_ylabel('% Mentioned')
    ax.set_xlabel('NDI Quartile')
    ax.set_yticklabels(['{:.0f}%'.format(x) for x in ax.get_yticks()])
    plt.legend(loc = 'center left',  bbox_to_anchor = (1, .5))
    plt.show()

for g in temp_groups:
    
    plot_chi_2(all_comps, g)    


In [ ]:
#save the results if you want to
save = False
if save:
    all_comps.to_csv(data_io.output_analysis/'chi2_tests_mention_by_NDI.csv')
    post_hoc.to_csv(data_io.output_analysis/'chi2_post_hoc_v1.csv')

In [ ]:
df.ndi_quantile_f.value_counts()

#### Run ANOVAs on NDI quartiles

In [ ]:
comparisons = ['raised_amnt', 'winsorized_goal', 'shares', 'num_contributors',
              'percent_goal', 'avg_contribution']
group_vals = np.unique(df['ndi_quantile_f'].values)
#Iterate over all comparisons
for c in comparisons:
    mod = ols(f'{c} ~ ndi_quantile_f',
                data=df).fit()
                
    aov_table = sm.stats.anova_lm(mod, typ=2)
    aov_table.reset_index(inplace=True)
    print(aov_table)
    multicomp = compare_info_by_group(df, [c], ['ndi_quantile_f'])
    
    for i in range(0, len(aov_table)):
        for col in aov_table.columns:
            multicomp.loc[i, col] = aov_table.loc[i, col]
            
    multicomp.rename(columns={'index':'omnibus'}, inplace=True)
    multicomp['outcome'] = c
    
    if c == comparisons[0]:
        all_multicomp = multicomp.copy()
    else:
        all_multicomp = pd.concat((all_multicomp, multicomp), ignore_index=True)

In [ ]:
#save results of ANOVA w post-hoc tests
if save:
    all_multicomp.to_csv(data_io.output_analysis/'anovas_by_NDI_quartile.csv', index=False)

#### Run comparisons in amount raised by potentially important confounding variables (year created, number of contributors, number of social media shares)

In [ ]:
#Compare outcomes by year
group_vals = np.unique(df['year'].values)

for c in comparisons:
    mod = ols(f'{c} ~ year',
                data=df).fit()
                
    aov_table = sm.stats.anova_lm(mod, typ=2)
    aov_table.reset_index(inplace=True)
    print(aov_table)
    multicomp = compare_info_by_group(df, [c], ['year'])
    
    for i in range(0, len(aov_table)):
        for col in aov_table.columns:
            multicomp.loc[i, col] = aov_table.loc[i, col]
            
    multicomp.rename(columns={'index':'omnibus'}, inplace=True)
    multicomp['outcome'] = c
    
    if c == comparisons[0]:
        mc_year = multicomp.copy()
    else:
        mc_year = pd.concat((mc_year, multicomp), ignore_index=True)

In [ ]:
raised_amount_year_info = df.groupby(['year']).agg({'raised_amnt': ['count', 'mean', 'std']})
#raised_amount_year_info.to_csv(data_io.output_analysis/'raised_amnt_by_year_desc.csv')


    
raised_amount_year_info.columns = ['count', 'mean', 'std']
raised_amount_year_info['mean_sd_fmt'] = raised_amount_year_info.apply(lambda x: format_mean_sd(x['mean'],
                                                                                               x['std']),
                                                                      axis = 1)
raised_amount_year_info


In [ ]:
if save:
    raised_amount_year_info.to_csv(data_io.output_analysis/'raised_amnt_by_year_desc.csv')


In [ ]:
#save results of ANOVA w post-hoc tests
if save:
    mc_year.to_csv(data_io.output_analysis/'anovas_by_campaign_year.csv', index=False)

In [ ]:
from scipy.stats import spearmanr

Get spearman rank correlations between amount raised and potential confounders

In [ ]:
corr_vars = ['num_contributors', 'shares', 'winsorized_goal']

temp_df = df[['num_contributors', 'shares', 'winsorized_goal', 'raised_amnt']]
corrs = temp_df.corr(method = 'spearman')
corrs = corrs[['raised_amnt']]

temp_df = df.dropna(subset=['raised_amnt'])

for i in range(0, len(corr_vars)):
    temp = temp_df.dropna(subset=[corr_vars[i]])
    r, p = spearmanr(temp[corr_vars[i]], temp['raised_amnt'])
    print(r, p)
    corrs.loc[corr_vars[i], 'p_value'] = p
if save:   
    corrs.to_csv(data_io.output_analysis/'raised_amnt_confound_corrs.csv')
corrs


#### Next, run t-tests on binary variables

In [ ]:
from sklearn.utils import resample
#From: https://campus.datacamp.com/courses/machine-learning-for-time-series-data-in-python/validating-and-inspecting-time-series-models?ex=12
def bootstrap_interval(data, percentiles=(2.5, 97.5), n_boots=100):
    """Bootstrap a confidence interval for the mean of columns of a 2-D dataset."""
    # Create empty array to fill the results
    bootstrap_means = np.zeros([n_boots, data.shape[-1]])
    for ii in range(n_boots):
        # Generate random indices for data *with* replacement, then take the sample mean
        random_sample = resample(data)
        bootstrap_means[ii] = random_sample.mean(axis=0)

    # Compute the percentiles of choice for the bootstrapped means
    percentiles = np.percentile(bootstrap_means, percentiles, axis=0)
    return percentiles


In [ ]:
def compare_two_groups(df, group_condition, group_col, group_labels,
                       comparison_cols = ['raised_amnt', 'winsorized_goal',
                                          'num_contributors', 'shares',
                                         'percent_goal', 'avg_contribution']):
    
    result_df= pd.DataFrame()
    g1 = df[df[group_col] == group_condition]
    g2 = df[df[group_col]!= group_condition]
    #Get t-test results for all comparison variables
    for i in range(0, len(comparison_cols)):
        #generate overall stats
        result_df.loc[i, 'percent_mentioned'] = format_percent(len(g1)/len(df))
        result_df.loc[i, 'overall_mean'] = df[comparison_cols[i]].mean()
        result_df.loc[i, 'overall_std_err'] = df[comparison_cols[i]].sem()
        result_df.loc[i, 'overall_sd'] = df[comparison_cols[i]].std()
        result_df.loc[i, 'comparison_var'] = comparison_cols[i]
        #Initialize temporary DF to store this comparison's results
        comp_result_df = pd.DataFrame()
        new_vnames = ['_g1', '_g2']
        #Set up a list of DFs to iterate over
        dfs = [g1, g2]
        k = 0
        for k in range(0, len(dfs)):
            temp_df = dfs[k].copy()
            comp_result_df.loc[0, 
                               f'n_missing{new_vnames[k]}'] = temp_df[comparison_cols[i]].isnull().sum()
            temp_df.dropna(subset = [comparison_cols[i]], inplace = True)
            comp_result_df.loc[0, f'n{new_vnames[k]}'] = len(temp_df)
            comp_result_df.loc[0, f'mean{new_vnames[k]}'] = temp_df[comparison_cols[i]].mean()
            comp_result_df.loc[0, f'fmtd_mean{new_vnames[k]}'] = format_decimal(temp_df[comparison_cols[i]].mean())
            comp_result_df.loc[0, f'std_err{new_vnames[k]}'] = temp_df[comparison_cols[i]].sem()
            comp_result_df.loc[0, f'sd{new_vnames[k]}'] = format_decimal(temp_df[comparison_cols[i]].std())
            comp_result_df.loc[0, f'name{new_vnames[k]}'] = group_labels[k]
            boot = bootstrap(temp_df[comparison_cols[i]].values)
            c_intervals = bootstrap_interval(temp_df[comparison_cols[i]])
            comp_result_df.loc[0, f'low_mean_ci{new_vnames[k]}'] = c_intervals[0, 0]
            comp_result_df.loc[0, f'up_mean_ci{new_vnames[k]}'] = c_intervals[1, 0]
            comp_result_df.loc[0, f'fmtd_mean_std{new_vnames[k]}'] = format_mean_sd(comp_result_df.loc[0, f'mean{new_vnames[k]}'],
                                                                                   comp_result_df.loc[0, f'sd{new_vnames[k]}'])
            #Replace old DF with temp DF to avoid null values when running t-test
            dfs[k] = temp_df
        
        #Run the t-test
        cm = sms.CompareMeans(sms.DescrStatsW(dfs[0][comparison_cols[i]]), 
                              sms.DescrStatsW(dfs[1][comparison_cols[i]]))
        #Format result as pandas DF
        comp_res = cm.summary()
        comp_res = comp_res.as_html()
        comp_res = pd.read_html(comp_res, header=0)[0]
        #Add this comparison's results to the overall result dataframe
        result_df.loc[i, 't_stat'] = comp_res.loc[0, 't']
        result_df.loc[i, 'p_val'] = comp_res.loc[0, 'P>|t|']
        result_df.loc[i, 'mean_diff'] = comp_res.loc[0, 'coef']
        result_df.loc[i, 'lower_ci'] = comp_res.loc[0, '[0.025']
        result_df.loc[i, 'upper_ci'] = comp_res.loc[0, '0.975]']
        temp_colnames = comp_result_df.columns.to_list()
        for t in temp_colnames:
            result_df.loc[i, t] = comp_result_df.loc[0, t]
    
    return result_df

def extract_varname(x):
    if type(x) == str:
        if ' is ' in x:
            split_loc = x.find(' is ')
            new_x = x[0:split_loc]
        else:
            new_x = x
        new_x = new_x.replace(" ", "\n")
        if '-' in new_x:
            new_x = "Self-\nReliance"
        return new_x

In [ ]:
group_label_dict = {}
for i in range(0, len(groups)):
    group_label_dict[groups[i]] = [pretty_groups[i] + 'is mentioned', pretty_groups[i] + 'is not mentioned']

In [ ]:
#Remove NDI from groups temporarily since they can't be subjected to a t-test    
temp_groups = ['self_reliance_is_mentioned', 'brave_is_mentioned', 'battle_is_mentioned', 'tx_type_is_mentioned',
               'oop_type_is_mentioned', 'insurance_type_is_mentioned', 'ctype_is_mentioned', 'nice_is_mentioned',
              'thank_is_mentioned']

In [ ]:


i = 0
for i in range(0, len(temp_groups)):
    print(group_label_dict[temp_groups[i]])
    temp = compare_two_groups(df, True, temp_groups[i], 
                                   group_label_dict[temp_groups[i]])
    temp['var_name'] = temp_groups[i]
    temp_fmtd_name = group_label_dict[temp_groups[i]][0]
    temp['fmtd_var_name'] = extract_varname(temp_fmtd_name)
    if i == 0:
        t_tests = temp.copy()
    else:
        t_tests = pd.concat([t_tests, temp], ignore_index= True)





In [ ]:
t_tests

In [ ]:
#save results of t-test
save = True
if save:
    t_tests.to_csv(data_io.output_analysis/'t_test_results.csv', encoding='utf-8', index=False)
    #t_tests.to_csv('t_tests_new_method.csv')


### Generate more detailed descriptives on GFM vars

In [ ]:

#Define functions for descriptive stats
def get_descriptive_stats(df):
    descriptives = {}
    cols_to_get = ['raised_amnt', 'winsorized_goal', 'num_contributors', 'avg_contribution',
                  'percent_goal', 'likes', 'shares']
    
    years = np.unique(df['year'].dropna().values)
    means = []
    sds = []
    medians = []
    mins = []
    maxes = []
    missing = []
    percent_missing = []
    mean_sd_fmt = []
    median_rg_fmt = []
    pct_miss_num = []
    
    stats = ['mean', 'sd', 'median', 'min', 'max', 'missing', 'percent_missing', 'mean_sd_fmt',
            'median_rg_fmt', 'pct_miss_num']
    
    for c in cols_to_get:
        #print(c)
        pct_miss_num.append(f"{format_percent(df[c].isnull().sum()/len(df))} ({format_decimal(df[c].isnull().sum())})")
        missing.append(format_decimal(df[c].isnull().sum()))
        percent_missing.append(format_percent(df[c].isnull().sum()/len(df)))
        if 'percent' not in c:
            means.append(format_decimal(df[c].mean()))
            sds.append(format_decimal(df[c].std()))
            medians.append(format_decimal(df[c].median()))
            mins.append(format_decimal(df[c].min()))
            maxes.append(format_decimal(df[c].max()))
            
            if 'raised' in c or 'winsor'in c or 'avg_con' in c: 
                mean_sd_fmt.append(f"${format_decimal(df[c].mean())} (${format_decimal(df[c].std())})")
                median_rg_fmt.append(f"${format_decimal(df[c].median())} (${format_decimal(df[c].min())} - ${format_decimal(df[c].max())})")
            else:
                mean_sd_fmt.append(f"{format_decimal(df[c].mean())} ({format_decimal(df[c].std())})")
                median_rg_fmt.append(f"{format_decimal(df[c].median())} ({format_decimal(df[c].min())} - {format_decimal(df[c].max())})")
            
        else:
            means.append(format_percent(df[c].mean()))
            sds.append(format_percent(df[c].std()))
            medians.append(format_percent(df[c].median()))
            mins.append(format_percent(df[c].min()))
            maxes.append(format_percent(df[c].max()))
            mean_sd_fmt.append(f"{format_percent(df[c].mean())} ({format_percent(df[c].std())})")
            median_rg_fmt.append(f"{format_percent(df[c].median())} ({format_percent(df[c].min())} - {format_percent(df[c].max())})")
    
    new = pd.DataFrame(columns=cols_to_get, index=stats, data = [means, sds, medians, mins, 
                                                                 maxes, missing, percent_missing,
                                                                    mean_sd_fmt, median_rg_fmt, pct_miss_num])
    
    #Non-numeric variables
    non_num = ['year', 'ndi_quantile_f', 'brave_is_mentioned', 'nice_is_mentioned', 'thank_is_mentioned', 
               'self_reliance_is_mentioned', 
                 'battle_is_mentioned', 'ctype_is_mentioned', 'oop_type_is_mentioned', 'insurance_type_is_mentioned', 
                 'tx_type_is_mentioned', 'met_goal']
    for n in non_num:
        val_counts = df[n].value_counts()
        for i in val_counts.index:
            index = str(i) +'_'+n
            new.loc['count', index] = val_counts[i]
            new.loc['percent', index] = format_percent(val_counts[i]/len(df))
            new.loc['fmt_count_pct', index] = f"{new.loc['percent', index]} ({val_counts[i]})"
    
    
    return new


In [ ]:
desc = get_descriptive_stats(df)

In [ ]:
desc

In [ ]:
if save:
    desc.to_csv(data_io.output_analysis/'gfm_campaign_descriptive_characteristics.csv', 
                encoding='utf-8')

### Multivariate Analysis

In [ ]:
#read in df with name mappings for formatting 
RE_FMT = pd.read_csv(data_io.input_cleaned/'gfm'/'rename_reg_outputs.csv')
def create_ref_vals(re_fmt):
    for i in re_fmt.index:
        if pd.isnull(re_fmt.loc[i, 'fmt_ref']) == False:
            re_fmt.loc[i, 'varname_w_ref'] = str(re_fmt.loc[i, 'fmt_varname_w_val']) + '\n' + re_fmt.loc[i, 'fmt_ref']
    return re_fmt

RE_FMT = create_ref_vals(RE_FMT)

#Functions to handle results of multivariate regression
def frmt_coef_and_se(coef, se):
    new_coef = format_decimal(coef, digits = 3)
    new_se = format_decimal(se, digits = 3)
    return f"{new_coef} ({new_se})"

def frmt_pct_and_ci(pct, ci_lower, ci_upper):
    return f"{pct}\n({ci_lower}, {ci_upper})"

#Reformats the regression outputs for easy plotting/reporting
def parse_results(result):
    results_summary = result.summary()
    #convert summary object to dataframe
    results_as_html = results_summary.tables[1].as_html()
    result_df = pd.read_html(results_as_html, header=0, index_col=0)[0]
    result_df.columns = ['coef', 'std err', 'z', 'p_value', 'coef_low_ci', 'coef_up_ci']
    result_df['percent_diff'] = np.exp((result_df['coef']))-1
    result_df['percent_diff_lower'] = np.exp((result_df['coef_low_ci']))-1
    result_df['percent_diff_upper'] = np.exp((result_df['coef_up_ci']))-1
    
    result_df['percent_diff_frmt'] = result_df.percent_diff.apply(format_percent)
    result_df['percent_diff_lower_frmt'] = result_df.percent_diff_lower.apply(format_percent)
    result_df['percent_diff_upper_frmt'] = result_df.percent_diff_upper.apply(format_percent)
    
    result_df['fmt_coef'] = result_df.apply(lambda x: frmt_coef_and_se(x['coef'], x['std err']),
                                           axis = 1)
    result_df['pct_ci_frmt'] = result_df.apply(lambda x: frmt_pct_and_ci(x['percent_diff_frmt'],
                                                                       x['percent_diff_lower_frmt'],
                                                                       x['percent_diff_upper_frmt']),
                                              axis = 1)
    
    result_df.reset_index(inplace=True)
    result_df.rename(columns={'index':'variable'}, inplace=True)
    temp_quantile_dict = {'ndi_quantile_f[T.2]': '2',
                      'ndi_quantile_f[T.3]': '3',
                      'ndi_quantile_f[T.4 (most deprived)]': '4 (most deprived)'}
    
    var_names = result_df['variable'].to_list()
    result_df['var_val'] = ''
    for i in range(0, len(var_names)):
        if 'T.True' in var_names[i]:
            var_names[i] = var_names[i].replace('[T.True]', '')
            result_df.loc[i, 'var_val'] = 'True'
        elif 'quantile' in var_names[i]:
            result_df.loc[i, 'var_val'] = temp_quantile_dict[var_names[i]]
            var_names[i] = 'ndi_quantile_f'


    result_df['var_name'] = var_names

    result_df['var_name_w_val'] = result_df['var_name'] + result_df['var_val'].astype(str)
    
    result_df = pd.merge(result_df, RE_FMT, how = 'left', left_on = 'var_name_w_val',
                        right_on = 'orig_varname_w_val')
    
    
    return result_df
    
    

In [ ]:
#Run the GLM
df['year_f'] = df['year'].astype(str)


raised_amnt = glm(formula = ('log_raised_amnt ~ ndi_quantile_f+ ctype_is_mentioned + brave_is_mentioned + '+
                            'winsorized_goal + shares + nice_is_mentioned + thank_is_mentioned+'+
                            'num_contributors + insurance_type_is_mentioned + self_reliance_is_mentioned +'+
                            'battle_is_mentioned + oop_type_is_mentioned + tx_type_is_mentioned + year_f'),
                  data = df, family = sm.families.Gaussian())
result = raised_amnt.fit()
results_summary = result.summary()

In [ ]:
#Format GLM results
reg_results = parse_results(result)
reg_results.loc[pd.isnull(reg_results['order']), 'order'] = 15
reg_results = reg_results.sort_values(by = 'order')
#save the results
#reg_results.to_csv(data_io.output_analysis/'glm_percent_change_formatted_v1.csv', index=False)
reg_results

In [ ]:
save = False
if save:
    reg_results.to_csv(data_io.output_analysis/'glm_percent_change.csv')

In [ ]:
#https://www.statisticshowto.datasciencecentral.com/wald-test/
wald = result.wald_test_terms()
wald.table

In [ ]:
#Add NDI quantiles back into the groups
if 'ndi_quantile_f' not in groups:
    groups.append('ndi_quantile_f')
    pretty_groups.extend(['NDI Quartile '])
else:
    print('ndi already in groups')

### Plot the results of the multivariate regression

In [ ]:
from matplotlib import rcParams
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:

#add significance/p-val indicators
def assign_significance(percent, sig):
    if pd.isnull(percent) == False:
        if sig <= 0.001:
            return percent + '***'
        elif sig <= 0.01:
            return percent + '**'
        elif sig <= 0.05:
            return percent + '*'
        else:
            return percent
def set_aspect_ratio(ax,ratio=0.5):
    xleft, xright = ax.get_xlim()
    ybottom, ytop = ax.get_ylim()
    # the abs method is used to make sure that all numbers are positive
    # because x and y axis of an axes maybe inversed.
    ax.set_aspect(abs((xright-xleft)/(ybottom-ytop))*ratio)
        


In [ ]:
#Set up plot for text characteristics
temp_means = t_tests.drop(columns = ['p_val'])
temp_means = temp_means[temp_means['comparison_var'] == 'raised_amnt']
merged = pd.merge(temp_means, reg_results, on = 'var_name')
merged['labels'] = merged.apply(lambda x: assign_significance(x['percent_diff_frmt'],
                                                        x['p_value']),
                                 axis = 1)
merged

In [ ]:

df.groupby('ndi_quantile_f').apply(lambda x: bootstrap_interval(x['raised_amnt']))



In [ ]:

#Set up plot for NDI 
ndi_metrics = df.groupby('ndi_quantile_f').agg({'raised_amnt': ['mean', 'std', 'sem']}).reset_index()
ndi_metrics.columns = ['var_val', 'mean_amnt_raised', 'std_amnt_raised', 'std_err_amnt_raised']
ndi_metrics['var_name'] = 'ndi_quantile_f'
for i in range(0, len(ndi_metrics)):
    temp_df = df[df['ndi_quantile_f'] == ndi_metrics.loc[i, 'var_val']]
    temp_df = temp_df.dropna(subset=['raised_amnt'])
    temp_ci = bootstrap_interval(temp_df['raised_amnt'])
    ndi_metrics.loc[i, 'lower_mean_ci'] = temp_ci[0, 0]
    ndi_metrics.loc[i, 'upper_mean_ci'] = temp_ci[1, 0]
    print(ndi_metrics)
    
temp_ndi = pd.merge(ndi_metrics, reg_results, on = ['var_name', 'var_val'])
add_idx = len(temp_ndi)
for c in ndi_metrics.columns:
    temp_ndi.loc[add_idx, c] = ndi_metrics.loc[0, c]
temp_ndi['fmt_var_val'] = temp_ndi['var_val'].apply(extract_varname)
temp_ndi.sort_values(by = ['mean_amnt_raised'], ascending = False, inplace=True)
temp_ndi['labels'] = temp_ndi.apply(lambda x: assign_significance(x['percent_diff_frmt'],
                                                                    x['p_value']),
                                    axis = 1)

In [ ]:
save = True
if save:
    merged.to_csv(data_io.output_analysis/'text_mention_outputs_for_plots.csv')
    temp_ndi.to_csv(data_io.output_analysis/'ndi_outputs_for_plots.csv')

In [ ]:
fig, (ax2, ax1) = plt.subplots(nrows = 2, figsize = (15, 15))
#Set top axis
temp_ndi['fmt_var_val'] = ['NDI: 1\n(least deprived)', 'NDI: 2',
                          'NDI: 3', 'NDI: 4\n(most deprived)']
#compute error bars
temp_ndi['lower_errors'] = temp_ndi['mean_amnt_raised'] - temp_ndi['lower_mean_ci']
temp_ndi['upper_errors'] = temp_ndi['upper_mean_ci'] - temp_ndi['mean_amnt_raised']

ax2.bar(temp_ndi['fmt_var_val'], temp_ndi['mean_amnt_raised'],
       yerr = [temp_ndi['lower_errors'], temp_ndi['upper_errors']],
       facecolor='c', ec = 'k', width = 0.3)

ax2.set_ylabel('Mean Amount Raised (USD)')
rects = ax2.patches
rect_labels = ['Ref.']
rect_labels.extend(temp_ndi['labels'].dropna().to_list())
for rect, label in zip(rects, rect_labels):
    width = rect.get_width()
    height = rect.get_height()
    
    y = rect.get_y() + rect.get_height() + 200
    x = rect.get_x() + 0.15
    ax2.text(x, y, label,
            ha='center', va='bottom')
    
ax2.set_ylim(0, max(temp_ndi['mean_amnt_raised'])+500)
ax2.set_xlim(-.5, 3.5)
ax2.text(-0.85, max(temp_ndi['mean_amnt_raised'])+500, 'A.', 
         ha = 'left', va = 'top', size = 20)
set_aspect_ratio(ax2)

#Set the bottom axis
x_vals = np.arange(0, len(merged))

merged['lower_error_bar_g1'] = merged['mean_g1'] - merged['low_mean_ci_g1']
merged['upper_error_bar_g1'] = merged['up_mean_ci_g1'] - merged['mean_g1']
width = 0.25
new_xvals_g1 = [x - width/2 for x in x_vals]
new_xvals_g2 = [x + width/2 for x in x_vals]
ax1.bar(new_xvals_g1, merged['mean_g1'], width, 
                    yerr = [merged['lower_error_bar_g1'], merged['upper_error_bar_g1']],
                    label = 'Text Feature Mentioned',
                    facecolor = 'seagreen',
                    ec = 'k')

rects = ax1.patches
rect_labels = merged['labels'].dropna().to_list()
for rect, label in zip(rects, rect_labels):
    width = rect.get_width()
    height = rect.get_height()
    
    y = rect.get_y() + rect.get_height() + 300
    x = rect.get_x() + 0.15
    ax1.text(x, y, label,
            ha='center', va='bottom')

merged['lower_error_bar_g2'] = merged['mean_g2'] - merged['low_mean_ci_g2']
merged['upper_error_bar_g2'] = merged['up_mean_ci_g2'] - merged['mean_g2']    

ax1.bar(new_xvals_g2, merged['mean_g2'], 
        width, label = 'Text Feature Not Mentioned',
        yerr = [merged['lower_error_bar_g2'], merged['upper_error_bar_g2']],
        facecolor = 'lightgrey',
        edgecolor = 'k')
    
ax1.set_xticks(x_vals)
ax1.set_xticklabels(merged['fmtd_var_name'])

ax1.set_ylim(bottom = 0, top = max(merged['mean_g1'])+1400)
ax1.set_ylabel('Mean Amount Raised (USD)')
ax1.text(-1.5, max(merged['mean_g1'])+1400, 'B.', 
         ha = 'left', va = 'top', size = 20)
set_aspect_ratio(ax1)
ax1.legend(loc = 'upper center')

plt.tight_layout()
plt.savefig(data_io.output_plots/'all_reg_outputs.png', dpi = 400, bbox_inches = 'tight')
plt.show()
